# Using Mask R-CNN to segment objects from images

Using a pre-trained model to detect and segment objects. 



Thanks to https://github.com/matterport/Mask_RCNN for the original code.

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!python3.6 gdrive/My\ Drive/Colab\ Notebooks/Mask_RCNN/samples/Hello.py

Hello world!!


In [ ]:
 !pip uninstall -y tensorflow

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Successfully uninstalled tensorflow-2.6.0


In [ ]:
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/Mask_RCNN
!pip install -r requirements3.txt

/content/gdrive/My Drive/Colab Notebooks/Mask_RCNN
     |████████████████████████████████| 411.0 MB 35 kB/s 
     |████████████████████████████████| 312 kB 69.1 MB/s 
     |████████████████████████████████| 2.9 MB 34.9 MB/s 
     |████████████████████████████████| 3.8 MB 41.2 MB/s 
     |████████████████████████████████| 503 kB 78.4 MB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 391 kB 62.1 MB/s 
     |████████████████████████████████| 154 kB 52.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=bd54de4b85c7d7995823ac05ad13c3e28a2b6fb30471c7b1b07b64ca992c0854
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: tensorflow-estimator
    Found existing ins

In [ ]:
#%tensorflow_version 2.x #use this to select the version of Tensorflow to uninstall

%tensorflow_version 1.x #use this to select the version of Tensorflow to run


`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x #use this to select the version of Tensorflow to run`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.



restart kernel here

In [ ]:
#%cd C:\Users\tinom\Mask_RCNN
    
#!pip install -r requirements4.txt

In [ ]:
#Restart kernel

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [ ]:
# Root directory of the project
#ROOT_DIR = os.path.abspath("../")
#ROOT_DIR = os.path.abspath("C:/Users/tinom/Mask_RCNN")
ROOT_DIR = os.path.abspath("gdrive//My Drive//Colab Notebooks//Mask_RCNN")

In [ ]:
os.chdir(ROOT_DIR)
os.getcwd()


'C:\\Users\\tinom\\Mask_RCNN\\mrcnn'

In [ ]:
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
#import model as modellib

from mrcnn import visualize

C:\Users\tinom\anaconda3\envs\EnvPython3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tinom\anaconda3\envs\EnvPython3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tinom\anaconda3\envs\EnvPython3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tinom\anaconda3\envs\EnvPython3.6\lib\site-

In [ ]:
# Import COCO config
#sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
#import coco

In [ ]:
# Import HBW config
sys.path.append(os.path.join(ROOT_DIR, "samples/BND/"))  # To find local version
import BND3

In [ ]:
%matplotlib inline 

# Directory to save logs and trained model
#MODEL_DIR = os.path.join(ROOT_DIR, "logs")

#MODEL_DIR = r"C:\Users\tinom\Mask_RCNN\logs\Colab_Model"
MODEL_DIR = os.path.join(ROOT_DIR, "Production models")

# Local path to trained weights file
#COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

MODEL_PATH = os.path.join(MODEL_DIR, "mask_rcnn_BND_0030.h5")

# Download COCO trained weights from Releases if needed
#if not os.path.exists(COCO_MODEL_PATH):
#    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
#IMAGE_DIR = os.path.join(ROOT_DIR, "images")

#IMAGE_DIR = "C://Users//tinom//Downloads//DissertationData//Dolphin"
IMAGE_DIR = "//content//gdrive//My Drive//Colab Notebooks//Mask_RCNN//datasets//BND//Train_100//val//Images"

## Configurations

We'll be using a model initially trained on the MS-COCO dataset then updated to learn whale fins using a training set of 100 images. The configurations of this model are in the ```HBWConfig``` class in ```HBW.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CoConfig``` class and override the attributes you need to change.

In [ ]:
class InferenceConfig(BND3.BNDConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## Create Model and Load Trained Weights

In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [ ]:
# Load weights trained on MS-COCO + annotated photographs
model.load_weights(MODEL_PATH, by_name=True)

## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [ ]:
# Load COCO dataset
dataset = BND3.BNDDataset()
BND_DIR = os.path.join(ROOT_DIR, "datasets//BND//Train_200")
dataset.load_BND(BND_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)

['BG', 'dolphin']


In [ ]:
class_names = ['BG', 'dolphin']

## Run Object Detection

In [ ]:
import cv2
#import argparse
import imutils
import colorsys

In [ ]:
#source = r"C:\Users\tinom\Downloads\DissertationData\Dolphin"
#destination = r"C:\Users\tinom\Mask_RCNN\BND\17112021_Cropped Images"

In [ ]:
source = os.path.join(ROOT_DIR, "datasets//BND//Dolphin")
destination = os.path.join(ROOT_DIR, "Output//17112021_Cropped Images")

Get files that haven't been cropped yet

In [ ]:
#Function to get order of masks from left to right

def order_masks(masks):
    
    if masks.shape[0] == 1:
        return [0]
    else:
        
        mask_left_most_point = []

        for i in range(masks.shape[0]):
            positions = np.nonzero(masks[i])
            left = positions[1].min()
            mask_left_most_point.append(left)

        array = np.array(mask_left_most_point)
        temp = array.argsort()
        ranks = np.empty_like(temp)
        ranks[temp] = np.arange(len(array))

        return ranks

In [ ]:
# check if some images have already been cropped, if so only rerurn the uncropped images

def check_cropped(source, target):
    
    ImageFiles = pd.DataFrame(os.listdir(source))

    if len(os.listdir(target)) > 0:
        CroppedFiles = pd.DataFrame(os.listdir(target))
        CroppedFiles =  pd.DataFrame(CroppedFiles[0].str.split("_", expand = True))
        RemainingImages = ImageFiles[0].loc[ImageFiles[0].isin(list(CroppedFiles[2])) == False]
    else: 
        RemainingImages = ImageFiles[0]
        
    return RemainingImages

In [ ]:
def crop_images(RemainingImages, source, destination):
    
    res = pd.DataFrame(columns = ["File", "Instances"])
    
    for img in tqdm(list(RemainingImages)):
        
        print("Processing image: " + img)
        OGimage = skimage.io.imread(os.path.join(source, img))

        # Run detection
        results = model.detect([OGimage], verbose=1)

        # Visualize results
        r = results[0]

        #visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
        #                       class_names, r['scores'])

         #visualize.display_instances2(OGimage, r['rois'], r['masks'], r['class_ids'], 
         #                   class_names, r['scores'], img = img)

        #Processed_image = visualize.display_instances2(OGimage, r['rois'], r['masks'], r['class_ids'], 
        #                    class_names, r['scores'], img = img)

        ## get and then save the segmented objects

        i = 0

        mask = r["masks"]

        reshaped_masks = mask.transpose(2,0,1).reshape(mask.shape[2], r["masks"].shape[0],r["masks"].shape[1])
        ranks = order_masks(reshaped_masks)
        
        newRes = pd.DataFrame([[img, len((r["rois"]))]], columns = ["File", "Instances"])
        #print(newRes)
        res = res.append(newRes, ignore_index = True) 
        #print(res)
        
        
        for i in range(r["masks"].shape[2]):
            OGimage = cv2.imread(os.path.join(source, img))

            OGimage = cv2.resize(OGimage, (r["masks"].shape[1], r["masks"].shape[0]))

            for j in range(OGimage.shape[2]):

                mask = cv2.dilate(np.float32(r["masks"]), cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (40,40)), iterations = 1)

                try:
                    mask1 = mask.transpose(2,0,1).reshape(r["masks"].shape[2], r["masks"].shape[0],r["masks"].shape[1])[i]
                except:
                    mask1 = mask

                OGimage[:,:,j] = OGimage[:,:,j] * mask1[:,:]

            filename = os.path.join(destination, "CroppedDil_" + str(ranks[i]) + "_"+ img)
            cv2.imwrite(filename,OGimage)
            i+=1
        
        #print(pd.DataFrame([[img, len((r["rois"]))]], columns = ["File", "Instances"]))
        #res.append(pd.DataFrame([[img, len((r["rois"]))]], columns = ["File", "Instances"])) 
        #print(res)
        
        #print(pd.DataFrame([[img, i]], columns = ["File", "Instances"]))
        #res.append(pd.DataFrame([[img, i]], columns = ["File", "Instances"])) 
        #print(res)
        
    return res


In [ ]:
def load_image(img, source):

    OGimage = skimage.io.imread(os.path.join(source, img))

    # Run detection
    results = model.detect([OGimage], verbose=1)

    # Visualize results
    r = results[0]

    #visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
    #                       class_names, r['scores'])

     #visualize.display_instances2(OGimage, r['rois'], r['masks'], r['class_ids'], 
     #                   class_names, r['scores'], img = img)

    visualize.display_instances2(OGimage, r['rois'], r['masks'], r['class_ids'], 
                        class_names, r['scores'], img = img)

    ## get and then save the segmented objects

    i = 0

    mask = r["masks"]

    reshaped_masks = mask.transpose(2,0,1).reshape(mask.shape[2], r["masks"].shape[0],r["masks"].shape[1])
    ranks = order_masks(reshaped_masks)

    print(masks.shape)

In [ ]:
AllFolders = os.listdir(source)
ExcludedFolders = ["Dolphin Data", "Images", "NO_ID"]

folders = list(set(AllFolders).difference(ExcludedFolders))
#folders = ["1","3","4"]


In [ ]:
res = pd.DataFrame(columns = ["File", "Instances"])

for subfolder in tqdm(folders):
    
    next if os.path.exists(os.path.join(destination, subfolder)) else os.mkdir(os.path.join(destination, subfolder))
    remaining_images = check_cropped(os.path.join(source, subfolder), os.path.join(destination, subfolder))
    res = res.append(crop_images(remaining_images, os.path.join(source, subfolder), os.path.join(destination, subfolder)))

  0%|                                                                                           | 0/71 [00:00<?, ?it/s]

Processing image: SB09-1260-333.JPG
Processing 1 images
image                    shape: (2592, 3888, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3888.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


  1%|█▏                                                                                 | 1/71 [00:17<20:11, 17.31s/it]

Processing image: SB09-1260-467.JPG
Processing 1 images
image                    shape: (2592, 3888, 3)       min:    8.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3888.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


  3%|██▎                                                                                | 2/71 [00:32<18:43, 16.28s/it]

...

Processing image: SB12-1466-634.JPG
Processing 1 images
image                    shape: (2304, 3456, 3)       min:   15.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  147.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3456.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


 37%|██████████████████████████████                                                    | 26/71 [06:01<10:22, 13.84s/it]

Processing image: SB12-1472-334.JPG
Processing 1 images
image                    shape: (2304, 3456, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3456.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


 38%|███████████████████████████████▏                                                  | 27/71 [06:10<09:08, 12.47s/it]

Processing image: SB12-1472-338.JPG
Processing 1 images
image                    shape: (2304, 3456, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3456.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


In [ ]:
res.to_csv (os.path.join(ROOT_DIR, "Results//171121_CroppingResults.csv"), index = False, header=True)

In [ ]:
crop_images(remaining_images, os.path.join(source, subfolder), os.path.join(destination, subfolder))

In [ ]:
list(remaining_images)

In [ ]:
subfolder = "1"

This section below crops the mask as is.

In [ ]:
# Loap through all images in selected folder

res = pd.DataFrame(columns = ["File", "Instances"])

for img in tqdm(list(RemainingImages)):
    
    #img = os.listdir(IMAGE_DIR)[i]
    
    file_names = img
    
    OGimage = skimage.io.imread(os.path.join(IMAGE_DIR, img))

    # Run detection
    results = model.detect([OGimage], verbose=1)

    # Visualize results
    r = results[0]

    #visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
    #                       class_names, r['scores'])

    res = res.append(pd.DataFrame([[img, len((r["rois"]))]], columns = ["File", "Instances"]))

    #visualize.display_instances2(OGimage, r['rois'], r['masks'], r['class_ids'], 
     #                   class_names, r['scores'], img = img)

    #Processed_image = visualize.display_instances2(OGimage, r['rois'], r['masks'], r['class_ids'], 
    #                    class_names, r['scores'], img = img)

    ## get and then save the segmented objects

    i = 0

    mask = r["masks"]
    
    reshaped_masks = mask.transpose(2,0,1).reshape(mask.shape[2], r["masks"].shape[0],-1)
    ranks = order_masks(reshaped_masks)
    
    for i in range(r["masks"].shape[2]):
        OGimage = cv2.imread(os.path.join(IMAGE_DIR, img))

        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #OGimage = imutils.resize(OGimage, width=r["masks"].shape[1], height = r["masks"].shape[0])
        OGimage = cv2.resize(OGimage, ( r["masks"].shape[1], r["masks"].shape[0]))

        for j in range(OGimage.shape[2]):

            mask = cv2.dilate(np.float32(r["masks"]), cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (28,28)), iterations = 1)
            
            try:
                mask1 = mask.transpose(2,0,1).reshape(2, r["masks"].shape[0],-1)[i]
            except:
                mask1 = mask

            OGimage[:,:,j] = OGimage[:,:,j] * mask1[:,:]

        filename = filename = os.path.join(CroppedDir, "CroppedDil_" + str(ranks[i]) + "_"+ img)
        cv2.imwrite(filename,OGimage)
        i+=1

   
    
res.to_csv (r'C:\Users\tinom\Mask_RCNN\BND\Instance segmentation on trained weights\Results2.csv', index = False, header=True)

# Loop through all images in selected folder

res = pd.DataFrame(columns = ["File", "Instances"])

for img in RemainingImages:
    
    #img = os.listdir(IMAGE_DIR)[i]
    
    file_names = img
    
    #try:
    OGimage = skimage.io.imread(os.path.join(IMAGE_DIR, img))

    # Run detection
    results = model.detect([OGimage], verbose=1)

    # Visualize results
    r = results[0]

    #visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
    #                       class_names, r['scores'])

    res = res.append(pd.DataFrame([[img, len((r["rois"]))]], columns = ["File", "Instances"]))

    #visualize.display_instances2(OGimage, r['rois'], r['masks'], r['class_ids'], 
     #                   class_names, r['scores'], img = img)

    #Turning off visualise to make cropping faster
    #Processed_image = visualize.display_instances2(OGimage, r['rois'], r['masks'], r['class_ids'], 
    #                    class_names, r['scores'], img = img)

    ## get and then save the segmented objects

    i = 0

    mask = r["masks"]

    for i in range(mask.shape[2]):
        OGimage = cv2.imread(os.path.join(IMAGE_DIR, img))

        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        OGimage = imutils.resize(OGimage, width=mask.shape[1])

        for j in range(OGimage.shape[2]):

              OGimage[:,:,j] = OGimage[:,:,j] * mask[:,:,i]

        filename = os.path.join(CroppedDir, "Cropped_" + str(i) + "_"+ img)
        cv2.imwrite(filename,OGimage)
        i+=1

  #  except:
   #     pass
    
#res.to_csv (r'C:\Users\tinom\Mask_RCNN\BND\Instance segmentation on trained weights\Results2.csv', index = False, header=True)

The section below pads the mask before cropping.

In [ ]:
res.append(pd.DataFrame([[img, len((r["rois"]))]], columns = ["File", "Instances"]))res.append(pd.DataFrame([[img, len((r["rois"]))]], columns = ["File", "Instances"]))


In [ ]:
a = pd.DataFrame(columns = ["Test", "Result"])
a

In [ ]:
b = pd.DataFrame([["Maths","2"]], columns = ["Test", "Result"])
b

In [ ]:
a.append(b)